In [12]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

import evaluate_results

In [13]:
def read_file(file_path):
    with open(file_path, "r", encoding="utf-8") as file:
        return file.read()

In [14]:
def compare_files(input_file_path, profile_file_path, output_file_path):
    #with open(input_file_path, 'r') as input_file:

    # compare files
    # extract personal data
    personal_data_dict = evaluate_results.extract_personal_data(profile_file_path)

    # read list of medication names from file
    medication_list = evaluate_results.read_medication_file()

    output_content = read_file(output_file_path)
    input_content = read_file(input_file_path)

    # evaluate structure
    structure_dict = evaluate_results.evaluate_structure(output_content)
    structure_score = evaluate_results.print_results(structure_dict, "structure", False)

    # evaluate correctness
    correctness_dict = evaluate_results.evaluate_correctness(structure_dict, personal_data_dict, medication_list,
                                                             input_content, output_content)
    correctness_score = evaluate_results.print_results(correctness_dict, "correctness", False)

    return structure_score, correctness_score
    #print(f"Comparing {input_file_path} and {profile_file_path} with {output_file_path}:")

In [30]:
result_file_path = "result.csv"

# create result file if it does not exist
if not os.path.isfile(result_file_path):
    # Define the base directory
    base_dir = "C:\\Users\\magda\\Documents\\Studium\\DSE\\MA\\Praxis\\Experimente"

    # Define the folder names
    folders = ["6", "7", "8", "9"]

    # Initialize an empty DataFrame with the specified structure
    columns = ['Prompt', 'Befundname', 'Anzahl Bsp', 'Str WH 1', 'Korr WH 1', 'Str WH 2', 'Korr WH 2', 'Str WH 3', 'Korr WH 3', 'Avg. Struktur', 'Avg. Korrektheit']
    df = pd.DataFrame(columns=columns)

    # Iterate through each folder
    for folder in folders:
        folder_path = os.path.join(base_dir, folder)

        # Check if the folder exists
        if os.path.exists(folder_path) and os.path.isdir(folder_path):
            # Paths to input.txt and profile.txt
            input_file_path = os.path.join(folder_path, "input.txt")
            profile_file_path = os.path.join(folder_path, "profile.txt")

            # Ensure input.txt and profile.txt exist
            if os.path.isfile(input_file_path) and os.path.isfile(profile_file_path):
                # List all files in the folder
                for filename in os.listdir(folder_path):
                    file_path = os.path.join(folder_path, filename)

                    # Check if it's an output file with the specific naming convention
                    if os.path.isfile(file_path) and filename.startswith("output-") and filename.endswith(".txt"):
                        parts = filename.split('-')
                        if len(parts) == 5:
                            dir_name = parts[1]
                            prompt_name = parts[2]
                            example_count = parts[3]
                            example_number = parts[4].split('.')[0]

                            if dir_name.isdigit() and prompt_name.isdigit() and example_count in ['1', '3', '5'] and example_number in ['1', '2', '3']:
                                if example_number == "1":  # save entries to df
                                    new_row = {
                                        "Prompt": prompt_name,
                                        "Befundname": dir_name,
                                        "Anzahl Bsp": example_count,
                                    }
                                    df.loc[len(df)] = new_row

                                # Define the condition to filter rows
                                condition = (df["Prompt"] == prompt_name) & (df["Befundname"] == dir_name) & (df["Anzahl Bsp"] == example_count)
                                # Find the index where the condition is true
                                index_to_update = df[condition].index

                                # get comparison results for structure and correctness
                                structure_result, correctness_result = compare_files(input_file_path, profile_file_path, file_path)

                                if not index_to_update.empty:
                                    structure_column_to_update = "Str WH " + example_number
                                    correctness_column_to_update = "Korr WH " + example_number

                                    # Update the value in column at the filtered index
                                    df.loc[index_to_update, structure_column_to_update] = structure_result
                                    df.loc[index_to_update, correctness_column_to_update] = correctness_result

                                #df = df.append(comparison_result, ignore_index=True)


    # Calculate averages for 'Avg. Struktur' and 'Avg. Korrektheit'
    df['Avg. Struktur'] = df[[f'Str WH {i}' for i in range(1, 4)]].mean(axis=1)
    df['Avg. Korrektheit'] = df[[f'Korr WH {i}' for i in range(1, 4)]].mean(axis=1)

    # Display the DataFrame
    #print(df)
    df.to_csv(result_file_path, sep=";", decimal=",")


In [31]:
# Load the CSV file
df = pd.read_csv(result_file_path, sep=";", decimal=",")

# Display the first few rows of the dataframe to understand its structure
df.head()

,Unnamed: 0,Prompt,Befundname,Anzahl Bsp,Str WH 1,Korr WH 1,Str WH 2,Korr WH 2,Str WH 3,Korr WH 3,Avg. Struktur,Avg. Korrektheit
0,0,1,6,1,90.0,50.0,100.0,70.0,90.0,50.0,93.333333,56.666667
1,1,1,6,3,90.0,50.0,90.0,50.0,90.0,50.0,90.000000,50.000000
2,2,1,6,5,100.0,60.0,100.0,70.0,100.0,60.0,100.000000,63.333333
3,3,2,6,1,90.0,80.0,100.0,60.0,100.0,80.0,96.666667,73.333333
4,4,2,6,3,100.0,70.0,100.0,80.0,90.0,80.0,96.666667,76.666667


In [32]:
# Calculate the averages of "Avg. Struktur" and "Avg. Korrektheit" grouped by the "Prompt" column
averages = df.groupby('Prompt')[['Avg. Struktur', 'Avg. Korrektheit']].mean().reset_index()
averages


,Prompt,Avg. Struktur,Avg. Korrektheit
0,1,93.333333,66.944444
1,2,97.777778,73.611111
2,3,95.833333,70.833333
3,4,92.500000,71.111111
4,5,90.833333,66.388889
5,6,73.055556,62.222222


In [33]:
# Calculate the averages of "Avg. Struktur" and "Avg. Korrektheit" grouped by the "Befundname" column
averages_befundname = df.groupby('Befundname')[['Avg. Struktur', 'Avg. Korrektheit']].mean().reset_index()

averages_befundname

,Befundname,Avg. Struktur,Avg. Korrektheit
0,6,94.259259,64.629630
1,7,87.592593,67.777778
2,8,87.222222,77.037037
3,9,93.148148,64.629630


In [34]:
# Calculate the averages of "Avg. Struktur" and "Avg. Korrektheit" grouped by the "Anzahl Bsp" column
averages_anzahl_bsp = df.groupby('Anzahl Bsp')[['Avg. Struktur', 'Avg. Korrektheit']].mean().reset_index()

averages_anzahl_bsp

,Anzahl Bsp,Avg. Struktur,Avg. Korrektheit
0,1,86.527778,68.194444
1,3,93.194444,70.694444
2,5,91.944444,66.666667
